In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
spark.sql("create or replace table datasets.practice.orders(order_id int, order_date date, customer_id int, order_status string) ")

In [0]:
spark.sql("show tables").display()

In [0]:
%sql
select * from datasets.practice.orders;

In [0]:
df = spark.read.format("csv")\
    .option("inferSchema", True)\
    .option("header", True)\
    .load("/Volumes/datasets/practice/internal_files/orders.csv")

In [0]:
df.display()

In [0]:
df2 = df.withColumn("order_date", to_timestamp(col("order_date"), "HH:mm.s"))

In [0]:
df2.printSchema()

In [0]:
df2.createOrReplaceTempView("orders")

In [0]:
%sql
select * from orders;

In [0]:
%sql
select * from datasets.practice.orders;

In [0]:
spark.sql("insert into datasets.practice.orders select * from orders")

In [0]:
%sql
select * from datasets.practice.orders;

In [0]:
%sql
select count(*) from datasets.practice.orders;

In [0]:
spark.sql("describe extended datasets.practice.orders").show(truncate=False)

#Drop and test the Managed Tables with all scenarios

In [0]:
%sql
select * from datasets.practice.orders

In [0]:
%sql
insert into table datasets.practice.orders values(100000, '2020-01-01',2001,'COMPLETED');

In [0]:
%sql
SELECT * FROM datasets.practice.orders WHERE customer_id = 2001;

#hERE Both Data and metadata is been dropped

In [0]:
%sql
DROP TABLE datasets.practice.orders;

In [0]:
%sql
SELECT * FROM datasets.practice.orders;

#External Table

###Here only the metadata is been dropped but the data resides in the cloud location as it is

In [0]:
%sql
CREATE TABLE IF NOT EXISTS datasets.practice.orders_ext (
    order_id INT,
    order_date DATE,
    customer_id INT,
    order_status STRING
)
USING CSV
OPTIONS (
    header 'true',
    inferSchema 'true'
)
LOCATION '/Volumes/datasets/practice/internal_files/orders.csv';


In [0]:
%sql
select * from workspace.gold.dimairports

In [0]:
%sql
desc table workspace.gold.dimairports

In [0]:
%sql
describe table workspace.gold.dimairports

In [0]:
%sql
describe extended workspace.gold.dimairports

In [0]:
spark.sql("describe extended workspace.gold.dimflights").display()

#🔄 Differences Between Managed and External Tables
| Feature             | **Managed Table**                                  | **External Table**                                    |
| ------------------- | -------------------------------------------------- | ----------------------------------------------------- |
| **Storage control** | Databricks controls the file location              | You define the file location (e.g., S3, ADLS, Volume) |
| **Drop behavior**   | Drops both metadata and data                       | Drops metadata only — data files remain               |
| **Create location** | Stored in DBFS or Unity-managed storage            | Stored in your provided path                          |
| **Use case**        | When Databricks manages the full lifecycle of data | When you want to reuse or share external data files   |
| **Example CREATE**  | `CREATE TABLE tbl`                                 | `CREATE TABLE tbl LOCATION 's3://my-bucket/path'`     |


#🛠 Operations Exclusive or More Common with Each
| Operation                               |     Managed    |         External        |
| --------------------------------------- | :------------: | :---------------------: |
| Auto location creation on `CREATE`      |        ✅       | ❌ (you must provide it) |
| Data deleted on `DROP TABLE`            |        ✅       |            ❌            |
| Can be registered over existing data    | ❌ (by default) |            ✅            |
| Used with Unity Catalog Volumes         |        ❌       |            ✅            |
| Reused by other systems (Hive, Athena)  |        ❌       |            ✅            |
| Better for sandbox/testing environments |        ✅       |            ❌            |
